## Unzip The Raw Data

In [3]:
!./unzip.sh UCI_HAR_Dataset.zip 2>&1 > /dev/null

## Building The Dataset

In [ ]:
import numpy as np
import glob
import os

def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])

def load_y(subset):
    # Get the path
    path = f'UCI_HAR_Dataset/UCI_HAR_Dataset/{subset}/y_{subset}.txt'

    # Read the file
    y = np.loadtxt(path, delimiter=',', dtype=int)









xs











    # # One-hot encode labels
    one_hot_labels = get_one_hot(y - 1 , len(np.unique(y)))
    
    assert one_hot_labels.shape == (7352, 6), f"Wrong dimensions: {one_hot_labels.shape} should be (7352, 6)"
    assert y[0] - 1 == np.where(one_hot_labels[0] == np.max(one_hot_labels[0]))[0][0], f"Value mismatch {np.max(one_hot_labels[0])[0][0]} vs {y[13] - 1}"
    return one_hot_labels

def load_data():
    return build_data('train'), load_y('train'), build_data('test'), load_y('test')

def build_data(subset):

    if subset not in ['train', 'val', 'test']:
        throw_error(f"Invalid subset: {subset}")

    folder_path = f"UCI_HAR_Dataset/UCI_HAR_Dataset/{subset}/Inertial tSignals/"
    # Get all signal files in folder
    signal_files = glob.glob(os.path.join(folder_path, '*.txt'))

    assert len(signal_files) > 0, f"No signal files found in {folder_path}"

    signal_shape = np.loadtxt(signal_files[0]).shape
    print(signal_shape)
    exit()

    # Determine signal order based on file names
    signal_order = [
        "body_acc_x_",
        "body_acc_y_",
        "body_acc_z_",
        "body_gyro_x_",
        "body_gyro_y_",
        "body_gyro_z_",
        "total_acc_x_",
        "total_acc_y_",
        "total_acc_z_",
        ]

    signal_order = [lambda x: x + f"{subset}.txt" for x in signal_order]

    # Initialize empty array
    combined_data = np.empty((7000, 128, len(signal_files)))

    for i, signal_file in enumerate(signal_order):
        if signal_file not in signal_files:
            throw_error(f"File not found: {signal_file}")

        # Load data from file into temporary array
        signal_data = np.loadtxt(signal_file)

        # Interleave data from temporary array into combined array
        combined_data[:, :, i%9] = signal_data  # i%9 ensures interleaving within range of 0 to 8
        combined_data[:, :, i%9 + 9] = signal_data  # interleaved copy

    # Remove extra interleaved copies
    combined_data = combined_data[:, :, :9]

    # Verify final shape of combined data
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    if subset == 'train':
        assert combined_data.shape == (7352, 128, len(signal_files))
    else:
        assert combined_data.shape == (2947, 128, len(signal_files))
    return combined_data

X_train, y_train, X_test, y_test = load_data()